# Finding analogues in the RADCLIM Zarr archive

### Abstract

This notebook describes how to find the analogues for a given date between the 1st January 1940 and the 31th December 2006 in the **RADCLIM-Analogs** dataset. 

This dataset provides maps at 1 km resolution of the hourly accumulated precipitation over Belgium from 1940 to 2016. We used the analog technique to provide for every day in the past (1940 – 2016) the 25 best analogs selected from the high resolution RMI RADCLIM radar database that is available from 2017 to 2022.

This notebook is provide as supplementary material for the article:

* Debrie, E., Demaeyer, J., and Vannitsem, S.: Hourly precipitation series over Belgium based on the Analogue Technique, Earth Syst. Sci. Data Discuss. [preprint], doi:, in review, 2025.

Before running this notebook, the dataset must first be downloaded from the Zenodo platform (https://zenodo.org/records/14712408) and installed following the installation instruction.





### Goal of the present notebook

In the present notebook, we are going to load analogues from the dataset for a given date, linking a database of analogue dates and the RADCLIM data.


### Code

Imports

In [ ]:
import os

In [ ]:
import xarray as xr

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

import cmocean.cm as cm

import cartopy

import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
import datetime

Some parameter variables:

In [ ]:
path_to_zarr = "./" # Where you unpacked the dataset.
fields_zarr_archive = "radclim.zarr"
analogs_zarr_archive = "precipitation-analogs.zarr"

Loading the RADCLIM Zarr archive

In [ ]:
ds = xr.open_zarr(os.path.join(path_to_zarr, zarr_archive_name))

In [ ]:
ds

Loading the Analogs database

In [ ]:
analogs_ds = xr.open_zarr(os.path.join(path_to_zarr, analogs_zarr_archive))

In [ ]:
analogs_ds

Defining a function that find the analogue fields based on a provided date

In [ ]:
def find_analogue_fields(isodate, numbers=None):
    """
    A function to find in the RADCLIM data the hourly-accumulated analogue fields for the provided date.

    Parameters
    ----------
    isodate: str
        The date for which the analogue fields must be returned, in the iso format: YYYY-MM-DD. Should be between 1940 and 2016 (included).
    numbers: list or None
        The number of the analogue fields to be returned, as a list. Should be a list of non-repeating numbers between 1 and 25.
        If `None`, return the full list of the 25 analogues.

    Returns
    -------
    dict(xarray.Dataset)
        The hourly-accumulated analogue fields for the asked selection of the analogue for the provided input date.
    """
    analog_dates = analogs_ds.analog_dates.sel(day=isodate) # get the list of analogue dates in the database
    almostaday = np.timedelta64(23, 'h') + np.timedelta64(59, 'm') # defined a timedelta of almost 24 hrs to selected all the fields of the day

    # if numbers was not provided, take all the analogues
    if numbers is None:
        numbers = list(range(1, len(analog_dates)+1))

    # get the analogues and put them in a dictionnary
    out_ds = dict()
    for n, date in zip(numbers, analog_dates.sel(number=numbers)):
        out_ds[n] = ds.sel(time=slice(date, date + almostaday))

    return out_ds
        

**Usage example:** Loading the first 4 analogs of the 14th of July 2010.

In [ ]:
analogs = find_analogue_fields('2010-07-14', numbers=[1,2,3,4])

Xarray Dataset of the first analogue

In [ ]:
analogs[1]

Plotting the daily accumulation (sum) of the 4 first analogues

In [ ]:
proj = ccrs.PlateCarree()
fig, axs = plt.subplots(2, 2, subplot_kw={'projection': proj},figsize=(14,8),)
axls = axs.flatten()
levels = np.linspace(0., 100., 201)

for i, n in enumerate(analogs):
    dss = analogs[n]
    ax = axls[i]
    im = ax.contourf(dss.longitude, dss.latitude, dss.tp.sum(dim='time'), transform=proj, levels=levels, cmap=cm.rain, extend='max')
    fig.colorbar(im, ax=ax)
    time_str = str(dss.time[0].to_numpy()).split('T')[0]
    ax.set_title(f'Analog {n}: {time_str}')
    ax.set_extent([2.5, 6.5, 49., 52.])
    ax.coastlines(resolution='50m')
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.gridlines(draw_labels={"bottom": "x", "left": "y"}, )

plt.tight_layout() 

The end.